# Planes VLM Eval — no cache, no file writes

Этот ноутбук — упрощённая версия пайплайна:

- **Без кэширования** и **без сохранения файлов** (CSV, PNG и т.п.).
- Вывод **поэтапно** прямо в ячейках Jupyter (таблица результатов обновляется в процессе, метрики считаются онлайн).
- Используется Hugging Face Inference Router в формате OpenAI API.

Перед запуском:
1. Установите зависимости (см. ячейку ниже).
2. Убедитесь, что у вас выставлена переменная окружения `HF_TOKEN` (например, через `.env`).
3. Укажите корректные пути `DATA_DIR` и, при наличии, `LABELS_CSV`.


In [1]:
%pip install -U python-dotenv openai pillow tqdm pandas scikit-learn matplotlib


Note: you may need to restart the kernel to use updated packages.


In [2]:
from __future__ import annotations
import os, json, base64, time, re, random
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
from pathlib import Path

from dotenv import load_dotenv
from openai import OpenAI
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

plt.rcParams['figure.figsize'] = (4,4)

# ---- Global label space ----
LABELS = ("civilian", "military")

@dataclass(frozen=True)
class VLMConfig:
    name: str
    provider: Optional[str] = None
    temperature: float = 0.0
    max_tokens: int = 200

def normalize_label(s: str) -> str:
    s = (s or "").strip().lower()
    if s in LABELS:
        return s
    if "civ" in s or "пасс" in s or "pass" in s:
        return "civilian"
    if "mil" in s or "арм" in s or "воен" in s or "army" in s:
        return "military"
    return ""

def collect_images(data_dir: Path) -> List[Path]:
    exts = {".jpg", ".jpeg", ".png", ".webp"}
    return sorted([p for p in data_dir.iterdir() if p.is_file() and p.suffix.lower() in exts])

def read_labels_csv(csv_path: Path) -> Dict[Path, str]:
    df = pd.read_csv(csv_path)
    base = csv_path.parent
    out: Dict[Path, str] = {}
    for _, row in df.iterrows():
        fn = base / str(row["filename"])  # ожидание колонок: filename,label
        out[fn.resolve()] = str(row["label"]) 
    return out

def infer_label_from_name(p: Path) -> str:
    s = p.stem.lower()
    if re.search(r"mil(it|itary)?", s) or "воен" in s or "army" in s:
        return "military"
    if re.search(r"civ(il|ilian)?", s) or "pass" in s or "пасс" in s:
        return "civilian"
    return ""

def load_ground_truth(data_dir: Path, labels_csv: Optional[Path]) -> Dict[Path, str]:
    imgs = collect_images(data_dir)
    labels = read_labels_csv(labels_csv) if labels_csv and labels_csv.exists() else {}
    gt: Dict[Path, str] = {}
    for p in imgs:
        lab = normalize_label(labels.get(p.resolve(), "") or infer_label_from_name(p))
        if lab in LABELS:
            gt[p] = lab
    if not gt:
        raise RuntimeError("Не найдено размеченных изображений (labels.csv или метка в имени файла).")
    return gt

def encode_image_to_data_url(path: Path) -> str:
    ext = path.suffix.lower().lstrip(".") or "jpeg"
    if ext not in {"jpg", "jpeg", "png", "webp"}:
        ext = "jpeg"
    b64 = base64.b64encode(path.read_bytes()).decode("utf-8")
    return f"data:image/{ext};base64,{b64}"

class HfVlmClassifier:
    def __init__(self, api_key: str, cfg: VLMConfig):
        self.client = OpenAI(base_url="https://router.huggingface.co/v1", api_key=api_key)
        self.model = f"{cfg.name}:{cfg.provider}" if cfg.provider else cfg.name
        self.temperature = cfg.temperature
        self.max_tokens = cfg.max_tokens

    def classify(self, image_path: Path, retries: int = 3, backoff: float = 0.8) -> Dict:
        img = encode_image_to_data_url(image_path)
        prompt = (
            "Classify the airplane in this photo strictly as 'civilian' or 'military'. "
            "Return JSON with fields: label, confidence (0..1), rationale."
        )
        last_err: Optional[Exception] = None
        for attempt in range(retries + 1):
            try:
                r = self.client.chat.completions.create(
                    model=self.model,
                    messages=[{
                        "role": "user",
                        "content": [
                            {"type": "text", "text": prompt},
                            {"type": "image_url", "image_url": {"url": img}},
                        ],
                    }],
                    response_format={"type": "json_object"},
                    temperature=self.temperature,
                    max_tokens=self.max_tokens,
                )
                msg = (r.choices[0].message.content or "{}").strip()
                msg = re.sub(r"^```(?:json)?\s*|\s*```$", "", msg)  # убираем обёртку ```json
                data = json.loads(msg)
                lab = normalize_label(data.get("label", ""))
                if not lab:
                    raise ValueError(f"Bad label: {data}")
                conf = float(data.get("confidence", 0))
                rat = str(data.get("rationale", ""))
                return {"label": lab, "confidence": conf, "rationale": rat}
            except Exception as e:
                last_err = e
                if attempt < retries:
                    time.sleep(backoff * (2 ** attempt))
        raise RuntimeError(f"Inference failed for {image_path.name}: {last_err}")

def evaluate(y_true: List[str], y_pred: List[str]) -> Tuple[float, float, float, float]:
    acc = accuracy_score(y_true, y_pred)
    p, r, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average="binary", pos_label="military", zero_division=0
    )
    return acc, p, r, f1

def plot_confusion(y_true: List[str], y_pred: List[str], title: str):
    cm = confusion_matrix(y_true, y_pred, labels=list(LABELS))
    fig = plt.figure()
    ax = plt.gca()
    im = ax.imshow(cm, interpolation="nearest")
    ax.set_xticks(range(len(LABELS))); ax.set_yticks(range(len(LABELS)))
    ax.set_xticklabels(LABELS); ax.set_yticklabels(LABELS)
    ax.set_xlabel("Predicted"); ax.set_ylabel("True"); ax.set_title(title)
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, str(cm[i, j]), ha="center", va="center")
    fig.colorbar(im)
    plt.tight_layout()
    plt.show()

def run_eval_for_model(cfg: VLMConfig, gt: Dict[Path, str], token: str, limit: Optional[int] = None, seed: int = 42):
    # Подготовка данных
    items = list(gt.items())
    if limit and len(items) > limit:
        random.seed(seed)
        items = random.sample(items, k=limit)

    clf = HfVlmClassifier(token, cfg)
    y_true, y_pred = [], []
    rows = []  # накопитель для отображения

    print(f"\n=== Model: {cfg.name}{' : ' + cfg.provider if cfg.provider else ''} ===")
    for idx, (path, true_lab) in enumerate(tqdm(items, desc="classify", unit="img"), start=1):
        pred = clf.classify(path)
        y_true.append(true_lab)
        y_pred.append(pred.get("label", ""))

        rows.append({
            "#": idx,
            "filename": path.name,
            "true": true_lab,
            "pred": pred.get("label", ""),
            "confidence": float(pred.get("confidence", 0)),
        })

        # Онлайн-метрики и таблица (обновляем вывод)
        acc, p, r, f1 = evaluate(y_true, y_pred)
        clear_output(wait=True)
        print(f"=== Model: {cfg.name}{' : ' + cfg.provider if cfg.provider else ''} ===")
        print(f"Processed: {idx}/{len(items)}")
        print(f"Accuracy: {acc:.3f}  |  Precision: {p:.3f}  Recall: {r:.3f}  F1: {f1:.3f}")
        df = pd.DataFrame(rows)
        display(df.tail(min(10, len(df))))  # последние 10 строк, чтобы вывод не разрастался

    # Финальные метрики + матрица ошибок
    print("\nFinal metrics:")
    acc, p, r, f1 = evaluate(y_true, y_pred)
    print(f"Accuracy: {acc:.4f}\nPrecision: {p:.4f}\nRecall: {r:.4f}\nF1: {f1:.4f}")
    plot_confusion(y_true, y_pred, title=cfg.name)


In [3]:
# === Параметры окружения и данных ===
load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")
if not HF_TOKEN:
    raise RuntimeError("Переменная окружения HF_TOKEN не найдена. Установите токен (например, через .env).")

# Укажите директорию с изображениями и (опционально) CSV с колонками: filename,label
DATA_DIR = Path("./planes")  # TODO: замените на вашу папку с картинками
LABELS_CSV = Path("./planes/labels.csv")  # Например: Path("/mnt/data/planes_dataset/labels.csv")

gt = load_ground_truth(DATA_DIR, LABELS_CSV)
print(f"Loaded images with labels: {len(gt)}")
print("Labels distribution:")
print(pd.Series(list(gt.values())).value_counts())


Loaded images with labels: 30
Labels distribution:
military    16
civilian    14
Name: count, dtype: int64


In [7]:
# === Модели для прогона ===
# Убедитесь, что выбранные модели доступны через ваш HF Router аккаунт.
MODELS = [
    VLMConfig(name="Qwen/Qwen2.5-VL-7B-Instruct"),
    VLMConfig(name="llava-hf/llava-v1.6-mistral-7b-hf", provider="replicate")
]
LIMIT = None  # можно уменьшить для быстрого прогрева, None — без лимита


In [9]:
for cfg in MODELS:
    run_eval_for_model(cfg, gt, token=HF_TOKEN, limit=LIMIT)



=== Model: Qwen/Qwen2.5-VL-7B-Instruct ===


classify:   0%|          | 0/30 [00:04<?, ?img/s]


KeyboardInterrupt: 